# Predicción usando Machine Learning clásico

Una vez los datos están limpios, ya podemos seleccionar y entrengar algunos modelos de Machine learning. Estamos ante un problema de predicción supervisado, por lo que usaremos algoritmos acordes a este tipo de problemas.

Al tratarse de un problema de regresión, tomaremos como medida de confianza el error cuadrático medio RMSE.

Estos son los modelos que probaremos:
- Regresión lineal
- Lasso
- Random forest

## Lectura dataset
Recuperamos los datasets de train y test que ya teníamos limpios.

In [1]:
import pandas as pd
import os
from sklearn import preprocessing

# Carga el dataset y devuelve un dataframe de Pandas
def load_airbnb_dataset(ruta,nombre):
    csv_path = os.path.join(ruta, nombre)
    return pd.read_csv(csv_path, sep=';')

In [2]:
copTrain = load_airbnb_dataset('datasets', 'train_final.csv')
copTest = load_airbnb_dataset('datasets', 'test_final.csv')


In [3]:
copTrain

,original_language,vote_average,budget,actor1,actor2,actor3,actor4,actor5,actor6,actor7,...,genero_actor27,genero_actor28,genero1,genero2,productora1,productora2,years_old,release_year,release_month,release_day
0,21.0,6.9,0.0,1547.0,3788.0,4564.0,4475.0,1501.0,1825.0,3217.0,...,0.0,0.0,1.0,4.0,3143.0,0.0,10,2010,5,17
1,9.0,5.1,10000000.0,801.0,1962.0,3370.0,2142.0,597.0,839.0,246.0,...,0.0,0.0,1.0,1.0,2657.0,15.0,23,1997,3,27
2,9.0,5.8,10000000.0,656.0,2491.0,2947.0,4570.0,4681.0,4698.0,2131.0,...,0.0,0.0,4.0,0.0,1158.0,644.0,13,2007,6,6
3,14.0,6.7,27440000.0,1285.0,2456.0,2751.0,4945.0,442.0,2020.0,303.0,...,0.0,0.0,4.0,4.0,3233.0,656.0,17,2003,10,22
4,9.0,6.2,18000000.0,2053.0,2022.0,2074.0,180.0,4577.0,1291.0,5979.0,...,0.0,0.0,12.0,13.0,2145.0,2596.0,17,2003,11,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7916,9.0,6.5,900000.0,2462.0,2409.0,173.0,926.0,574.0,3948.0,161.0,...,0.0,0.0,11.0,12.0,2658.0,468.0,17,2003,1,30
7917,9.0,6.6,0.0,1921.0,2150.0,1297.0,2830.0,3364.0,2159.0,242.0,...,0.0,0.0,4.0,13.0,1994.0,1343.0,13,2007,9,9
7918,9.0,8.4,185000000.0,968.0,2878.0,4765.0,3184.0,2018.0,390.0,1028.0,...,2.0,1.0,1.0,4.0,745.0,1461.0,12,2008,7,16
7919,9.0,7.4,5000000.0,212.0,3256.0,1735.0,3694.0,5614.0,5112.0,1257.0,...,1.0,2.0,4.0,13.0,136.0,937.0,3,2017,3,30


## Escalado de características

In [4]:
train_vote_average = copTrain['vote_average']
test_vote_average = copTest['vote_average']

train_sin_y = copTrain.drop(['vote_average'], axis=1, inplace=False)
test_sin_y = copTest.drop(['vote_average'], axis=1, inplace=False)

y_train = train_vote_average.values
X_train = train_sin_y.values

y_test = test_vote_average.values
X_test = test_sin_y.values

feature_names = copTrain.columns[:]

In [5]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)

scaler = preprocessing.StandardScaler().fit(X_test)
XtestScaled = scaler.transform(X_test)

## Seleccionar y entrenar modelo
Una vez normalizados los datos ya estamos preparados para seleccionar un modelo y entrenarlo.

### Modelo de regresión lineal

In [6]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(XtrainScaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#### Calcular la raíz del error cuadrático medio

In [7]:
from sklearn.metrics import mean_squared_error
import numpy as np

precicciones = lin_reg.predict(XtrainScaled)
prediccionesTest = lin_reg.predict(XtestScaled)

lin_mse = mean_squared_error(y_train, precicciones)
lin_rmse = np.sqrt(lin_mse)

lin_mseTest = mean_squared_error(y_test, prediccionesTest)
lin_rmseTest = np.sqrt(lin_mseTest)

print('Error train:', lin_rmse)
print('Error test:', lin_rmseTest)

Error train: 1.2560836631340253
Error test: 1.3466398267522337


Vemos que la raíz del error cuadrático medio nos da 1,25 en train y 1,34 en test, si recordamos, en la parte de exploración de los datos, vimos que la media de valoración nos daba 6,23. Así que nuestro modelo de regresión calcula un error medio de +/- 1,34 puntos.

Este error en porcentaje lo traduciríamos a un error del 21%, algo alto, luego intentaremos reducirlo con otro modelo.

Vamos a calcular también el **mae** (mean absolute error) por tener una medida más. Al mae no hay que calcularle la raíz cuadrada porque el resultado tiene la misma escala que los datos, al contrario que el error cuadrático medio.

In [8]:
from sklearn.metrics import mean_absolute_error

mae_train = mean_absolute_error(y_train, precicciones)
mae_test = mean_absolute_error(y_test, prediccionesTest)

print('Error train:', mae_train)
print('Error test:', mae_test)

Error train: 0.8385522843011168
Error test: 0.8855410165445805


### Lasso
Probamos ahora con Lasso usando además GridSearch para obtener los mejores parámetros.

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

model = Lasso()
params = {'alpha': [1, 0.1, 0.01, 0.001, 0.0001]}

grid = GridSearchCV(model, params, cv=10, iid=False)
grid.fit(XtrainScaled, y_train)

grid.best_estimator_.get_params()

D:\Programas\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'alpha': 0.01,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

#### Calculamos de nuevo el error cuadrático medio para compararlo con la regresión

In [10]:
alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(XtrainScaled,y_train)

ytrainLasso = lasso.predict(XtrainScaled)
ytestLasso  = lasso.predict(XtestScaled)

lin_mseLasso = mean_squared_error(y_train,ytrainLasso)
lin_rmseLasso = np.sqrt(lin_mseLasso)

lin_mseTestLasso = mean_squared_error(y_test,ytestLasso)
lin_rmseTestLasso = np.sqrt(lin_mseTestLasso)

print('Error train:', lin_rmseLasso)
print('Error test:', lin_rmseTestLasso)

Error train: 1.2578897057910614
Error test: 1.348262102551534


Hemos obtenido más o menos el mismo error que con la regresión, muestro ahora la importancia de las características:

In [11]:
w = lasso.coef_
for f,wi in zip(feature_names,w):
    print(f,wi)

original_language 0.0489652141963462
vote_average 0.0002298159286867367
budget -0.0
actor1 0.0031757194761233244
actor2 -0.0
actor3 0.0
actor4 0.028944992114828975
actor5 0.01999680598975813
actor6 0.021312472182696947
actor7 0.023353234686559237
actor8 0.040712183080650485
actor9 0.01657321634560078
actor10 0.023078414404324413
actor11 0.02807781818317215
actor12 0.030597488562271978
actor13 0.014329242117186725
actor14 0.0
actor15 0.013918252515727762
actor16 0.03931271595371843
actor17 0.013656148506863591
actor18 0.0
actor19 0.04044683397286164
actor20 0.0
actor21 0.007421457695908521
actor22 0.0017611778696178027
actor23 0.0
actor24 0.0
actor25 0.028911570919135
actor26 0.004694403807983316
actor27 0.028015320656636738
actor28 0.07349711274473353
genero_actor1 0.025200807414422127
genero_actor2 0.0205241009627549
genero_actor3 0.009339885317107239
genero_actor4 -0.0081803851385002
genero_actor5 -0.0
genero_actor6 0.0
genero_actor7 -0.0
genero_actor8 0.0
genero_actor9 0.0
genero_ac

#### Random forest

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

rfr = RandomForestRegressor(n_estimators=500, max_depth=2, random_state=0)
rfr.fit(X_train, y_train)

y_pred_rfr = rfr.predict(X_test)

predictions = cross_val_predict(rfr, X_test, y_test, cv=10)
print('Predicciones:', predictions)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred_rfr))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred_rfr))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred_rfr)))

Predicciones: [6.38794221 6.38794221 6.41842858 ... 6.3804624  7.09184016 6.3804624 ]
Mean Absolute Error: 0.7618047504500629
Mean Squared Error: 1.1839406497839597
Root Mean Squared Error: 1.0880903683904015


Si nos fijamos en la raíz del error cuadrático medio, el error ha pasado de 1,34 a 1.08, traducido a porcentaje, hemos pasado de un 21% a un 17% con el Random Forest.